In [17]:
import tensorflow as tf
import keras
from keras.models import Sequential
from keras.layers import Dense

In [18]:
import numpy as np
import pandas as pd


In [19]:
df=pd.read_csv('/content/WineQT.csv')
df.head()

,fixed acidity,volatile acidity,citric acid,residual sugar,chlorides,free sulfur dioxide,total sulfur dioxide,density,pH,sulphates,alcohol,quality,Id
0,7.4,0.70,0.00,1.9,0.076,11.0,34.0,0.9978,3.51,0.56,9.4,5,0
1,7.8,0.88,0.00,2.6,0.098,25.0,67.0,0.9968,3.20,0.68,9.8,5,1
2,7.8,0.76,0.04,2.3,0.092,15.0,54.0,0.9970,3.26,0.65,9.8,5,2
3,11.2,0.28,0.56,1.9,0.075,17.0,60.0,0.9980,3.16,0.58,9.8,6,3
4,7.4,0.70,0.00,1.9,0.076,11.0,34.0,0.9978,3.51,0.56,9.4,5,4


In [20]:
df['quality'].unique()

array([5, 6, 7, 4, 8, 3])

In [21]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1143 entries, 0 to 1142
Data columns (total 13 columns):
 #   Column                Non-Null Count  Dtype  
---  ------                --------------  -----  
 0   fixed acidity         1143 non-null   float64
 1   volatile acidity      1143 non-null   float64
 2   citric acid           1143 non-null   float64
 3   residual sugar        1143 non-null   float64
 4   chlorides             1143 non-null   float64
 5   free sulfur dioxide   1143 non-null   float64
 6   total sulfur dioxide  1143 non-null   float64
 7   density               1143 non-null   float64
 8   pH                    1143 non-null   float64
 9   sulphates             1143 non-null   float64
 10  alcohol               1143 non-null   float64
 11  quality               1143 non-null   int64  
 12  Id                    1143 non-null   int64  
dtypes: float64(11), int64(2)
memory usage: 116.2 KB


Splitting into Training and Validation Data

In [22]:
train_df=df.sample(frac=0.75,random_state=4)

In [23]:
train_df.shape

(857, 13)

In [24]:
train_df.index

Int64Index([ 667,  221,  265,  154,  517,  616,   98,  325,  963,  659,
            ...
              10, 1127,   21,  861,  637,  173,   38,  985,  757,  469],
           dtype='int64', length=857)

In [25]:
val_df=df.drop(train_df.index)

Min-Max scaling

In [26]:
max_val = train_df.max(axis=0)
min_val = train_df.min(axis=0)
range = max_val-min_val
train_df=(train_df-min_val)/range
val_df=(val_df-min_val)/range

In [27]:
X_train=train_df.drop('quality',axis=1)
X_val=val_df.drop('quality',axis=1)
Y_train=train_df['quality']
Y_val=val_df['quality']

input_shape=[X_train.shape[1]]
input_shape

[12]

In [29]:
model=tf.keras.Sequential([tf.keras.layers.Dense(units=64, activation='relu',input_shape=input_shape),tf.keras.layers.Dense(units=32,activation='relu'),tf.keras.layers.Dense(units=1)])
model.summary()

Model: "sequential"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 dense (Dense)               (None, 64)                832       
                                                                 
 dense_1 (Dense)             (None, 32)                2080      
                                                                 
 dense_2 (Dense)             (None, 1)                 33        
                                                                 
Total params: 2,945
Trainable params: 2,945
Non-trainable params: 0
_________________________________________________________________


In [30]:
model.compile(optimizer='adam', # or sgd can be used as optimizer
             # MAE is good for
             # numerical predictions
             # cross entropy for classification
             loss='mae')


In [32]:
losses = model.fit(X_train, Y_train,validation_data=(X_val, Y_val),
                  batch_size=100,
                  epochs=15,
                  )

Epoch 1/15
9/9 [==============================] - 0s 22ms/step - loss: 0.1033 - val_loss: 0.1011
Epoch 2/15
9/9 [==============================] - 0s 9ms/step - loss: 0.1027 - val_loss: 0.1023
Epoch 3/15
9/9 [==============================] - 0s 9ms/step - loss: 0.1023 - val_loss: 0.1003
Epoch 4/15
9/9 [==============================] - 0s 9ms/step - loss: 0.1012 - val_loss: 0.0994
Epoch 5/15
9/9 [==============================] - 0s 9ms/step - loss: 0.1010 - val_loss: 0.0999
Epoch 6/15
9/9 [==============================] - 0s 8ms/step - loss: 0.1005 - val_loss: 0.0984
Epoch 7/15
9/9 [==============================] - 0s 10ms/step - loss: 0.1002 - val_loss: 0.0984
Epoch 8/15
9/9 [==============================] - 0s 8ms/step - loss: 0.0990 - val_loss: 0.0977
Epoch 9/15
9/9 [==============================] - 0s 8ms/step - loss: 0.0992 - val_loss: 0.0987
Epoch 10/15
9/9 [==============================] - 0s 9ms/step - loss: 0.0986 - val_loss: 0.0979
Epoch 11/15
9/9 [====================

In [33]:
type(X_train)

pandas.core.frame.DataFrame

In [34]:
X_val.iloc[121:124,:]

,fixed acidity,volatile acidity,citric acid,residual sugar,chlorides,free sulfur dioxide,total sulfur dioxide,density,pH,sulphates,alcohol,Id
516,0.309735,0.410959,0.09,0.130137,0.120401,0.253731,0.151943,0.685022,0.594828,0.233533,0.415385,0.455116
519,0.159292,0.510274,0.03,0.157534,0.098662,0.388060,0.183746,0.361968,0.750000,0.095808,0.661538,0.456999
528,0.327434,0.438356,0.29,0.226027,0.105351,0.164179,0.035336,0.472100,0.612069,0.209581,0.476923,0.463905


In [35]:
Y_val[121:124]

516    0.6
519    0.6
528    0.6
Name: quality, dtype: float64

In [37]:
test=X_val.iloc[120]

test.shape

new_test = pd.DataFrame(test.values.reshape(1, -1))

In [38]:
model.predict(new_test)

1/1 [==============================] - 0s 347ms/step


array([[0.52368397]], dtype=float32)

In [39]:
Y_train.iloc[120]

0.6

In [40]:
X_val.iloc[200:210]

,fixed acidity,volatile acidity,citric acid,residual sugar,chlorides,free sulfur dioxide,total sulfur dioxide,density,pH,sulphates,alcohol,Id
817,0.327434,0.328767,0.25,0.089041,0.177258,0.119403,0.113074,0.447137,0.353448,0.119760,0.215385,0.722536
818,0.442478,0.205479,0.35,0.082192,0.118729,0.238806,0.113074,0.451542,0.422414,0.197605,0.415385,0.723164
821,0.044248,0.267123,0.18,0.082192,0.050167,0.223881,0.335689,0.171072,0.620690,0.323353,0.692308,0.725675
823,0.389381,0.455479,0.24,0.054795,0.110368,0.134328,0.053004,0.502937,0.474138,0.203593,0.246154,0.730069
827,0.407080,0.164384,0.34,0.047945,0.083612,0.059701,0.021201,0.484581,0.396552,0.203593,0.323077,0.733836
833,0.088496,0.544521,0.00,0.082192,0.048495,0.238806,0.254417,0.248899,0.810345,0.239521,0.461538,0.738858
835,0.460177,0.184932,0.43,0.051370,0.093645,0.059701,0.017668,0.345815,0.387931,0.077844,0.461538,0.740741
842,0.230088,0.089041,0.37,0.109589,0.085284,0.149254,0.123675,0.317181,0.672414,0.281437,0.615385,0.747646
843,0.212389,0.428082,0.12,0.061644,0.170569,0.208955,0.204947,0.426579,0.413793,0.155689,0.169231,0.748901
844,0.141593,0.212329,0.22,0.061644,0.110368,0.298507,0.176678,0.459618,0.672414,0.161677,0.169231,0.749529


In [41]:
Y_val.iloc[200:210]

817    0.4
818    0.6
821    0.8
823    0.4
827    0.6
833    0.4
835    0.4
842    0.8
843    0.6
844    0.6
Name: quality, dtype: float64

In [42]:
preds=model.predict(X_val.iloc[200:210,:])

1/1 [==============================] - 0s 23ms/step


In [43]:
arr_rounded = np.round(preds, decimals=1)

In [44]:
arr_rounded

array([[0.4],
       [0.6],
       [0.7],
       [0.5],
       [0.6],
       [0.6],
       [0.6],
       [0.7],
       [0.4],
       [0.4]], dtype=float32)

In [45]:
loss_df=pd.DataFrame(losses.history)

loss_df

,loss,val_loss
0,0.103341,0.101086
1,0.102686,0.102283
2,0.102321,0.100290
3,0.101226,0.099352
4,0.100997,0.099881
5,0.100474,0.098383
6,0.100166,0.098416
7,0.098963,0.097726
8,0.099222,0.098655
9,0.098599,0.097862


Let's check evaluation metrics for the small no. of prediction we created

In [46]:
import numpy as np
from sklearn.metrics import mean_absolute_error, mean_squared_error, r2_score

pred = arr_rounded
actual = Y_val.iloc[200:210]

mae = mean_absolute_error(actual, pred)
print("MAE:", mae)


mse = mean_squared_error(actual, pred)
print("MSE:", mse)


rmse = np.sqrt(mse)
print("RMSE:", rmse)
r2 = r2_score(actual, pred)
print("R^2 Score:", r2)

MAE: 0.1100000113248825
MSE: 0.0190000019073489
RMSE: 0.13784049443958368
R^2 Score: 0.15178562913620985


Evaluating the model on the whole dataset

In [48]:
preds=model.predict(X_val)

9/9 [==============================] - 0s 2ms/step


In [49]:
arr_rounded = np.round(preds, decimals=1)

In [50]:
pred = arr_rounded
actual = Y_val

mae = mean_absolute_error(actual, pred)
print("MAE:", mae)


mse = mean_squared_error(actual, pred)
print("MSE:", mse)

rmse = np.sqrt(mse)
print("RMSE:", rmse)


r2 = r2_score(actual, pred)
print("R^2 Score:", r2)

MAE: 0.09265734663793256
MSE: 0.01688811146296005
RMSE: 0.12995426681321415
R^2 Score: 0.2605823973748632
